## RockPaperScissorsLizardSpock.js

In this notebook we will finetune a SqueezeNet pretrained on ImageNet to classify between the classes 'rock', 'paper', 'scissors', 'lizard', 'spock' and 'other'.

### Data setup

There should already be a folder for each class in `./data/train/`. E.g. all `rock` images should be in `./data/train/rock`

In [1]:
import os
import numpy as np

In [2]:
data_dir = './data/'
model_dir = data_dir + 'models/'
train_dir = data_dir + 'train/'
val_dir = data_dir + 'val/'

The output of the next cell should be (in any order) `['lizard', 'spock', 'other', 'paper', 'scissors', 'rock']`

In [3]:
classes = os.listdir(train_dir)
num_classes = len(classes)
classes

['lizard', 'spock', 'other', 'paper', 'scissors', 'rock']

In [ ]:
os.makedirs(model_dir, exist_ok=True)

Create a train/validation split. 

In [19]:
os.makedirs(val_dir, exist_ok=True)

In [20]:
for c in categories:
    os.makedirs(val_dir + c, exist_ok=True)

In [21]:
def make_val_split(p):
    """
    Move random images of each category from train to val.
    p is the percentage of images to move, e.g.
    p == 0.2 will move 20% of each category to val.
    """
    for c in categories:
        file_names = os.listdir(train_dir + c)
        permutation = np.random.permutation(file_names)
        for i in range(int(len(file_names)*p)):
            file_path = os.path.join(c, permutation[i])
            os.rename(os.path.join(train_dir, file_path), 
                      os.path.join(val_dir, file_path))

In [22]:
# only do this once
make_val_split(0.2)

### Create and train the model

In [4]:
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, Dropout, Activation, GlobalAveragePooling2D
from keras.models import Model

Using TensorFlow backend.


We will use SqueezeNet but replace its top layers (the classification layers) to classify between 'rock', 'paper', 'scissors', 'lizard', 'spock' and 'other' and not between the 1000 ImageNet classes.

In [5]:
def get_model():
    base_model = SqueezeNet(weights='imagenet', include_top=False, input_shape=(227,227,3))
    x = base_model.output
    x = Dropout(0.5, name='drop9')(x)
    x = Convolution2D(num_classes, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)

    predictions = Activation('softmax', name='predictions')(x)

    return Model(inputs=base_model.input, outputs=predictions)

In [6]:
model = get_model()

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 113, 113, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 113, 113, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 56, 56, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [7]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [8]:
batch_size = 128

In [38]:
def get_batches(dirname, generator=ImageDataGenerator(), shuffle=True,
                batch_size=batch_size, class_mode='categorical', target_size=(227,227)):
    return generator.flow_from_directory(dirname, shuffle=shuffle, batch_size=batch_size,
                                 class_mode=class_mode, target_size=target_size)

The ImageDataGenerator will do standard imagenet preprocessing and some data augmentation.

In [39]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                rotation_range=15, 
                                height_shift_range=0.1, 
                                shear_range=0.1,
                                zoom_range=0.1,
                                width_shift_range=0.1)
train_batches = get_batches(train_dir, generator=train_gen)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_batches = get_batches(val_dir, generator=val_gen)

Found 3972 images belonging to 6 classes.
Found 990 images belonging to 6 classes.


In [14]:
num_train = train_batches.samples
num_val = val_batches.samples

In [40]:
def fit(model, epochs):
    model.fit_generator(train_batches, 
                        steps_per_epoch=num_train // batch_size,
                        epochs=epochs,
                        validation_data=val_batches,
                        validation_steps=num_val // batch_size)

In [41]:
model.optimizer.lr = 0.0001
fit(model, 5)

Epoch 1/5
31/31 [==============================] - 36s - loss: 1.7441 - acc: 0.3042 - val_loss: 1.3875 - val_acc: 0.5145
Epoch 2/5
31/31 [==============================] - 30s - loss: 1.0110 - acc: 0.6380 - val_loss: 0.5433 - val_acc: 0.8158
Epoch 3/5
31/31 [==============================] - 31s - loss: 0.5068 - acc: 0.8211 - val_loss: 0.2548 - val_acc: 0.9062
Epoch 4/5
31/31 [==============================] - 30s - loss: 0.3099 - acc: 0.8966 - val_loss: 0.1268 - val_acc: 0.9576
Epoch 5/5
31/31 [==============================] - 30s - loss: 0.2171 - acc: 0.9273 - val_loss: 0.0902 - val_acc: 0.9721


In [42]:
fit(model, 15)

Epoch 1/15
31/31 [==============================] - 35s - loss: 0.1852 - acc: 0.9331 - val_loss: 0.0724 - val_acc: 0.9810
Epoch 2/15
31/31 [==============================] - 30s - loss: 0.1272 - acc: 0.9579 - val_loss: 0.0558 - val_acc: 0.9821
Epoch 3/15
31/31 [==============================] - 30s - loss: 0.1130 - acc: 0.9609 - val_loss: 0.0705 - val_acc: 0.9754
Epoch 4/15
31/31 [==============================] - 31s - loss: 0.0842 - acc: 0.9710 - val_loss: 0.0198 - val_acc: 0.9933
Epoch 5/15
31/31 [==============================] - 30s - loss: 0.0540 - acc: 0.9821 - val_loss: 0.0123 - val_acc: 0.9955
Epoch 6/15
31/31 [==============================] - 31s - loss: 0.0538 - acc: 0.9816 - val_loss: 0.0222 - val_acc: 0.9944
Epoch 7/15
31/31 [==============================] - 30s - loss: 0.0555 - acc: 0.9795 - val_loss: 0.0174 - val_acc: 0.9933
Epoch 8/15
31/31 [==============================] - 31s - loss: 0.0471 - acc: 0.9841 - val_loss: 0.0123 - val_acc: 0.9978
Epoch 9/15
31/31 [======

In [43]:
model.optimizer.lr = 1e-6
fit(model, 15)

Epoch 1/15
31/31 [==============================] - 36s - loss: 0.0188 - acc: 0.9929 - val_loss: 0.0090 - val_acc: 0.9989
Epoch 2/15
31/31 [==============================] - 30s - loss: 0.0199 - acc: 0.9945 - val_loss: 0.0086 - val_acc: 0.9978
Epoch 3/15
31/31 [==============================] - 30s - loss: 0.0144 - acc: 0.9955 - val_loss: 0.0059 - val_acc: 0.9978
Epoch 4/15
31/31 [==============================] - 31s - loss: 0.0166 - acc: 0.9942 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 5/15
31/31 [==============================] - 30s - loss: 0.0165 - acc: 0.9952 - val_loss: 6.5666e-04 - val_acc: 1.0000
Epoch 6/15
31/31 [==============================] - 30s - loss: 0.0292 - acc: 0.9894 - val_loss: 0.0026 - val_acc: 1.0000
Epoch 7/15
31/31 [==============================] - 30s - loss: 0.0647 - acc: 0.9798 - val_loss: 0.4092 - val_acc: 0.9208
Epoch 8/15
31/31 [==============================] - 30s - loss: 0.1888 - acc: 0.9432 - val_loss: 0.0145 - val_acc: 0.9944
Epoch 9/15
31/31 [==

In [44]:
model.optimizer.lr = 5e-8
fit(model, 15)

Epoch 1/15
31/31 [==============================] - 35s - loss: 0.0081 - acc: 0.9977 - val_loss: 0.0041 - val_acc: 0.9989
Epoch 2/15
31/31 [==============================] - 30s - loss: 0.0063 - acc: 0.9987 - val_loss: 7.1949e-04 - val_acc: 1.0000
Epoch 3/15
31/31 [==============================] - 30s - loss: 0.0086 - acc: 0.9977 - val_loss: 0.0036 - val_acc: 0.9989
Epoch 4/15
31/31 [==============================] - 31s - loss: 0.0063 - acc: 0.9975 - val_loss: 7.6032e-04 - val_acc: 1.0000
Epoch 5/15
31/31 [==============================] - 30s - loss: 0.0114 - acc: 0.9967 - val_loss: 0.0040 - val_acc: 0.9989
Epoch 6/15
31/31 [==============================] - 29s - loss: 0.0129 - acc: 0.9965 - val_loss: 0.0159 - val_acc: 0.9967
Epoch 7/15
31/31 [==============================] - 30s - loss: 0.0085 - acc: 0.9980 - val_loss: 0.0095 - val_acc: 0.9978
Epoch 8/15
31/31 [==============================] - 31s - loss: 0.0084 - acc: 0.9967 - val_loss: 0.0059 - val_acc: 0.9989
Epoch 9/15
31/31

Save the weights for the case they are needed again.

In [45]:
model.save_weights(data_dir + 'models/model.hdf5')

To load them do
```python
model.load_weights(data_dir + 'models/model.hdf5')
```

### Export model and weights for deeplearn.js

In [10]:
import tensorflow as tf
from keras import backend as K
from keras.models import Model

The deeplearn.js repository provides the script `dump_checkpoint_vars.py` which takes a tensorflow checkpoint and converts it for use with deeplearn.js. To use this, we will first save the model as a tensorflow checkpoint.

In deeplearn.js this model will only be used for inference, no training will be done. So we have to tell keras to use the model in 'inference mode' where some layers behave differently (e.g. Dropout).

In [11]:
K.set_learning_phase(0)

The model has to be rebuilt for this to take effect:

In [12]:
config = model.get_config()
weights = model.get_weights()

model = Model.from_config(config)
model.set_weights(weights)

Save the checkpoint:

In [13]:
sess = K.get_session()
saver = tf.train.Saver()
saver.save(sess, model_dir + 'model.ckpt')

'./data/models/model.ckpt'

Use a slightly modified version of the provided script to convert the checkpoint:

In [14]:
%run dump_checkpoint_vars.py --output_dir=deeplearn-checkpoint \
--checkpoint_file=data/models/model.ckpt \
--remove_variables_regex training*|Adam*

Writing variable fire6/expand3x3_1/bias as fire6/expand3x3/bias
Writing variable fire6/expand3x3_1/kernel as fire6/expand3x3/kernel
Writing variable fire7/expand1x1_1/bias as fire7/expand1x1/bias
Writing variable fire7/expand1x1_1/kernel as fire7/expand1x1/kernel
Writing variable fire3/expand1x1_1/bias as fire3/expand1x1/bias
Writing variable fire3/expand1x1_1/kernel as fire3/expand1x1/kernel
Writing variable fire4/squeeze1x1_1/bias as fire4/squeeze1x1/bias
Writing variable fire4/squeeze1x1_1/kernel as fire4/squeeze1x1/kernel
Writing variable fire7/expand3x3_1/bias as fire7/expand3x3/bias
Writing variable fire7/expand3x3_1/kernel as fire7/expand3x3/kernel
Writing variable fire2/expand1x1_1/bias as fire2/expand1x1/bias
Writing variable fire2/expand1x1_1/kernel as fire2/expand1x1/kernel
Writing variable fire2/squeeze1x1_1/bias as fire2/squeeze1x1/bias
Writing variable fire2/squeeze1x1_1/kernel as fire2/squeeze1x1/kernel
Writing variable conv1_1/bias as conv1/bias
Writing variable conv1_1

This should have created the folder `deeplearn-checkpoint` in the current directory.